# Let's build the GPT tokenizer

## Introduction

Effects of tokenization on decoder LLM capabilities :
- unability to spell words
- unability to do basic arithmetics
- unability to do basic string processing tasks
- preference over some file format (e.g. YAML is better generated than JSON for GPT-2)
- preference over language -> less occuring language in the tokenizer training dataset have a longer token representation, which means the information is less dense for a given input from the model perspective.

From [Python documentation](https://docs.python.org/3/library/functions.html#ord), `ord` is the inverse function of `chr`. It means that its input are single Unicode characters, and its outputs are the associated Unicode code point (i.e integers). 

In [1]:
ord("h")

104

In [2]:
ord("😛")

128539

In [1]:
my_string = "おはよう！ It means 'hello' in Japanese :)"
[ord(char) for char in my_string]

[12362,
 12399,
 12424,
 12358,
 65281,
 32,
 73,
 116,
 32,
 109,
 101,
 97,
 110,
 115,
 32,
 39,
 104,
 101,
 108,
 108,
 111,
 39,
 32,
 105,
 110,
 32,
 74,
 97,
 112,
 97,
 110,
 101,
 115,
 101,
 32,
 58,
 41]

Unicode code points can't be used as such because they are too many. Instead, we can resort to using a unicode encoding such as [UTF-8](https://fr.wikipedia.org/wiki/UTF-8) which encodes each unicode symbol on 1 to 4 bytes, meaning any sentence would be representing by a sequence of integers ranging from 0 to 255.

### *Further readings* : 

#### [A programmer's introduction to Unicode](https://www.reedbeta.com/blog/programmers-intro-to-unicode/) and [UTF-8 everywhere](https://utf8everywhere.org/)

Unicode first draft proposal dates back to 1988, published py Joseph D. Becker. One interesting feature of UTF-8 is that it is *endianness independent* (independ of byte order).

*UTF* (as in *UTF-8*) stands for *Unicode Transformation Format*. The Unicode *codespace* is the set of all possible Unicode *codepoints*. The codepoints are the integer identifiers for each assigned symbols, with a potential range from 0 to 1,114,111 (with only approximately 12% assigned). Each code point could be represented with 4 bytes, but this reveals to be wasteful for various reasons, one of them being that most used Unicode symbols have codepoints inferior to 65,536. The 3 main Unicode encodings are :
- **UTF-8**: a *variable-length* encoding where symbols can be represented with 1 to 4 bytes depending on their codepoint
- **UTF-16**: a *variable-length* encoding where symbols can be represented with 2 to 4 bytes depending on their codepoint
- **UTF-32**: a *fixed-length* encoding where symbols are represented with exactly 4 bytes

We call *code unit* the minimal number of bits that can be used to represent a unit of encoded text (8 bits for UTF-8, 16 for UTF-16 and 32 for UTF-32).

The main advantage of variable-length encodings is memory saving, and their main drawback is that they are slower to compute.

For UTF-8 (and UTF-16 likewise), how can we distinguish between single and multi-bytes sequences ? Sequences of different bytes-length use unique prefixes:

| UTF-8 (binary)                      | Code point (binary)   | Range                               |
|-------------------------------------|-----------------------|-------------------------------------|
| 0xxxxxxx                            | xxxxxxx               | U+0000–U+007F (0-127)               |
| 110xxxxx 10yyyyyy                   | xxxxxyyyyyy           | U+0080–U+07FF (128-2,047)           |
| 1110xxxx 10yyyyyy 10zzzzzz          | xxxxyyyyyyzzzzzz      | U+0800–U+FFFF (2,048-65,535)        |
| 11110xxx 10yyyyyy 10zzzzzz 10wwwwww | xxxyyyyyyzzzzzzwwwwww | U+10000–U+10FFFF (65,536-1,114,111) |

Unicode symbols can be dynamically composed to form new characters (for instance, diacritics can be composed with letters). However, "composed" characters can have different representations resulting in the same rendering, and some composed characters even have their own codepoint in the Unicode standard! Hence the need for *normalization forms*, which reorder bytes according to certain rules, preserving the rendered characters but allowing for byte-to-byte comparison.

A *grapheme cluster* is a "user-perceived character"; this notion is particularly useful in the context of text editors, since they are used for cursor placement and text selection boundaries.

In [2]:
my_utf8_string = my_string.encode("utf-8")
list(my_utf8_string)

[227,
 129,
 138,
 227,
 129,
 175,
 227,
 130,
 136,
 227,
 129,
 134,
 239,
 188,
 129,
 32,
 73,
 116,
 32,
 109,
 101,
 97,
 110,
 115,
 32,
 39,
 104,
 101,
 108,
 108,
 111,
 39,
 32,
 105,
 110,
 32,
 74,
 97,
 112,
 97,
 110,
 101,
 115,
 101,
 32,
 58,
 41]

## Byte pair encoding

Why do we need byte pair encoding ? Language models have a limited context window in which a token can attend to all previous tokens, and utf-8 representation of text strings are necessarily at least as long as the text's number of characters. Byte pair encoding allows to find recurring patterns in a given "training" corpus, in order to add those patterns to the vocabulary, and thus reducing the number of tokens required to represent an average piece of text. In other words, byte pair encodind allows for statisticlly efficient information compression.